# Read WFS data from into Shapely/Cartopy

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import geojson
from owslib.wfs import WebFeatureService
from shapely.geometry import Polygon, mapping, asShape, shape
import cartopy.crs as ccrs
from cartopy.io.img_tiles import MapQuestOpenAerial, MapQuestOSM, OSM
%matplotlib inline

In [2]:
# Read data from WFS 1.1 service in JSON format
# note the WFS implementation has to offer JSON output format.

In [3]:
# getCapabilities
# {endpoint}?service=wfs&version=1.0.0&request=GetCapabilities
# each feature has a LatLongBoundingBox

In [4]:
# some Geoserver WFS endpoints:
#endpoint='https://www.sciencebase.gov/catalogMaps/mapping/ows/5342c54be4b0aa151574a8dc'
endpoint='https://www.sciencebase.gov/catalogMaps/mapping/ows/5342c5fce4b0aa151574a8ed'
#endpoint='https://www.sciencebase.gov/catalogMaps/mapping/ows/5342e124e4b0aa151574a969'
# for testing, doesn't support JSON output format:
#endpoint='http://services.azgs.az.gov/arcgis/services/aasggeothermal/AZActiveFaults/MapServer/WFSServer'
#smu thermal conductivity
#endpoint='http://geothermal.smu.edu:9000/geoserver/aasg-thermalconductivity/ows'
wfs = WebFeatureService(endpoint, version='1.1.0')
print wfs.version

1.1.0


In [5]:
featuretypes = wfs.contents.keys()
print featuretypes

['sb:footprint', 'sb:Conservation_Zone_WGS84']


In [6]:
a = wfs.contents[featuretypes[0]]
b = a.boundingBoxWGS84

In [7]:
shp = filter(lambda a: a != 'sb:footprint', featuretypes)
print shp

['sb:Conservation_Zone_WGS84']


In [8]:
def flip_geojson_coordinates(geo):
    if isinstance(geo, dict):
        for k, v in geo.iteritems():
            if k == "coordinates":
                z = np.asarray(geo[k])
                f = z.flatten()
                geo[k] = np.dstack((f[1::2], f[::2])).reshape(z.shape).tolist()
            else:
                flip_geojson_coordinates(v)
    elif isinstance(geo, list):
        for k in geo:
            print 'list key {0}'.format(k)
            flip_geojson_coordinates(k)

In [14]:
#get list of output formats. assumes getFeature is always the third operation in sequence
[wfs.operations[2].parameters['outputFormat']][0]['values']

['text/xml; subtype=gml/3.1.1',
 'GML2',
 'KML',
 'SHAPE-ZIP',
 'application/gml+xml; version=3.2',
 'application/json',
 'application/vnd.google-earth.kml xml',
 'application/vnd.google-earth.kml+xml',
 'csv',
 'gml3',
 'gml32',
 'json',
 'text/javascript',
 'text/xml; subtype=gml/2.1.2',
 'text/xml; subtype=gml/3.2']

In [ ]:
#srs='EPSG:4326' # v1.0 syntax
srs='urn:x-ogc:def:crs:EPSG:4326'  # v1.1 syntax
#srs='urn:ogc:def:crs:EPSG:6.9:4326' #ESRI? syntax
json_response = wfs.getfeature(typename=[shp[0]], propertyname=None, srsname=srs, outputFormat='application/json',maxFeatures='10').read()
geo = geojson.loads(json_response)
print 'geo done'
flip_geojson_coordinates(geo)
print 'flip done'

In [ ]:
print geo.keys()

In [ ]:
print geo['type']

In [ ]:
geodetic = ccrs.Geodetic(globe=ccrs.Globe(datum='WGS84'))

plt.figure(figsize=(12,12))
# Open Source Imagery from MapQuest (max zoom = 16?) [SMR2018-05-18 doesn't work]
#tiler = MapQuestOpenAerial()
# Open Street Map (max zoom = 18?)
tiler = OSM()
ax = plt.axes(projection=tiler.crs)
dx=b[2]-b[0]
dy=b[3]-b[1]
extent = (b[0]-0.1*dx,b[2]+0.1*dx,b[1]-0.1*dy,b[3]+0.1*dy)
ax.set_extent(extent, geodetic)
ax.add_image(tiler, 14)
#ax.add_geometries([polygon],ccrs.PlateCarree(),
#                          facecolor=BLUE, edgecolor=GRAY,alpha=0.5)
for p in geo.get("features", []):
    multi_poly = asShape(p.get("geometry"))
    print 'bounds from Shapely: ',multi_poly.bounds
#    name=p['properties']['NAME']
#    print name
    ax.add_geometries(multi_poly,ccrs.PlateCarree(),
                edgecolor='black',facecolor='none',hatch='/')
#title(name)
    
gl=ax.gridlines(draw_labels=True)
gl.xlabels_top = False
gl.ylabels_right = False
#ax.add_feature(coast_10m,edgecolor='black')
#ax.coastlines()